In [6]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium import Element
import os

CLEANED_DATA = '../data/cleaned/grocery_stores_cleaned_v3.csv'
COMMUNITY_GEOJSON = '../docs/community_areas.geojson'
MAP_OUTPUT = '../docs/grocery_stores_chicago_map_v1.html'
INIT_LOC = [41.8781, -87.6298]
INIT_ZOOM = 11

# 1️⃣ LOAD DATA
df = pd.read_csv(CLEANED_DATA)
df['IS_REAL_GROCERY'] = df['IS_REAL_GROCERY'].astype(bool)
df['IS_JUNK_STORE'] = df['IS_JUNK_STORE'].astype(bool)

areas = gpd.read_file(COMMUNITY_GEOJSON)
areas = areas.rename(columns={'community':'Community'})

# 2️⃣ INIT MAP
m = folium.Map(location=INIT_LOC, zoom_start=INIT_ZOOM)


# Add community area polygons
def style_area(feature):
    return {'fill': False, 'color': '#444444', 'weight': 1}

geo_json = folium.GeoJson(
    areas,
    name='🏘️ Community Areas',
    style_function=style_area,
    tooltip=folium.GeoJsonTooltip(fields=['Community'], aliases=['Area:']),
    highlight_function=lambda f: {'weight':2, 'color':'blue'}
).add_to(m)

# Inject JS to bind click-to-zoom for each polygon
script = f"""
<script>
  {geo_json.get_name()}.eachLayer(function(layer) {{
    layer.on('click', function() {{
      map.fitBounds(layer.getBounds());
    }});
  }});
</script>
"""
m.get_root().html.add_child(Element(script))

# Set up store clusters (click-to-zoom + spiderfy)
cluster_opts = {
    'zoomToBoundsOnClick': True,
    'spiderfyOnMaxZoom':  True,
    'showCoverageOnHover': False,
    'disableClusteringAtZoom': 30
}

real_layer = folium.FeatureGroup(name='🟢 Real Grocery Stores')
junk_layer = folium.FeatureGroup(name='🔴 Junk Stores')

real_cluster = MarkerCluster(options=cluster_opts).add_to(real_layer)
junk_cluster = MarkerCluster(options=cluster_opts).add_to(junk_layer)

# Add store markers
for _, row in df.iterrows():
    loc   = [row['Latitude'], row['Longitude']]
    popup = f"<b>{row['DBA Name']}</b><br>{row['Address']}, {row['City']} {row['Zip']}"

    if row['IS_REAL_GROCERY']:
        color, icon, cluster = 'green', 'shopping-cart', real_cluster
    elif row['IS_JUNK_STORE']:
        color, icon, cluster = 'red', 'remove', junk_cluster
    else:
        color, icon, cluster = 'gray', 'question'

    folium.Marker(
        location=loc,
        popup=popup,
        icon=folium.Icon(color=color, icon=icon, prefix='fa')
    ).add_to(cluster)


for layer in (real_layer, junk_layer):
    layer.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

os.makedirs(os.path.dirname(MAP_OUTPUT), exist_ok=True)
m.save(MAP_OUTPUT)
print(f"✅ Interactive store map with community-click zoom saved to: {MAP_OUTPUT}")

✅ Interactive store map with community-click zoom saved to: ../docs/grocery_stores_chicago_map_v1.html
